# Análise das interações dos estudantes em um MOOC

In [1]:
!pip install pyspark


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
!pip install pandas

     ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
      --------------------------------------- 0.2/10.7 MB 3.1 MB/s eta 0:00:04
     - -------------------------------------- 0.5/10.7 MB 5.4 MB/s eta 0:00:02
     ---- ----------------------------------- 1.2/10.7 MB 9.7 MB/s eta 0:00:01
     -------- ------------------------------- 2.2/10.7 MB 11.9 MB/s eta 0:00:01
     ------------ --------------------------- 3.3/10.7 MB 14.2 MB/s eta 0:00:01
     ----------------- ---------------------- 4.7/10.7 MB 16.8 MB/s eta 0:00:01
     ---------------------- ----------------- 5.9/10.7 MB 18.0 MB/s eta 0:00:01
     ------------------------- -------------- 6.9/10.7 MB 19.1 MB/s eta 0:00:01
     ------------------------------ --------- 8.1/10.7 MB 19.1 MB/s eta 0:00:01
     ---------------------------------- ----- 9.2/10.7 MB 19.6 MB/s eta 0:00:01
     ------------------------------------ -- 10.0/10.7 MB 20.0 MB/s eta 0:00:01
     --------------------------------------  10.6/1


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
!pip install matplotlib

     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
      --------------------------------------- 0.1/7.6 MB 2.2 MB/s eta 0:00:04
     -- ------------------------------------- 0.5/7.6 MB 5.0 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.6 MB 8.8 MB/s eta 0:00:01
     ---------- ----------------------------- 2.1/7.6 MB 11.1 MB/s eta 0:00:01
     -------------- ------------------------- 2.9/7.6 MB 12.2 MB/s eta 0:00:01
     --------------------- ------------------ 4.2/7.6 MB 15.6 MB/s eta 0:00:01
     --------------------------- ------------ 5.3/7.6 MB 16.8 MB/s eta 0:00:01
     ---------------------------------- ----- 6.6/7.6 MB 18.4 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.6 MB 18.7 MB/s eta 0:00:01
     ---------------------------------------- 7.6/7.6 MB 18.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/98.3 kB ? eta -:--:--
     ---------------------------------------- 98.3/98.3 kB ? eta 0:00:


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Coleta e processamento dos dados

In [16]:
# leitura dos dados
# df_2015_1 = spark.read.json("C:/Users/mulle/downloads/data/20150801-20151101-raw_user_activity.json")
# df_2015_2 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20151101-20160201-raw_user_activity.json")
# df_2016_1 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160201-20160501-raw_user_activity.json")

In [17]:
# df_2016_2 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160501-20160801-raw_user_activity.json")
# df_2016_3 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160801-20170201-raw_user_activity.json")
# df_2017_1 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20170201-20170801-raw_user_activity.json")

In [8]:
df_train = spark.read.csv('data/train_log.csv', header=False)
df_train = df_train.withColumnRenamed("_c0", "enroll_id")
df_train = df_train.withColumnRenamed("_c1", "username")
df_train = df_train.withColumnRenamed("_c2", "course_id")
df_train = df_train.withColumnRenamed("_c3", "session_id")
df_train = df_train.withColumnRenamed("_c4", "action")
df_train = df_train.withColumnRenamed("_c5", "object")
df_train = df_train.withColumnRenamed("_c6", "time")
df_train = df_train.where(df_train.username != 'username')

df_test = spark.read.csv('data/test_log.csv', header=False)
df_test = df_test.withColumnRenamed("_c0", "enroll_id")
df_test = df_test.withColumnRenamed("_c1", "username")
df_test = df_test.withColumnRenamed("_c2", "course_id")
df_test = df_test.withColumnRenamed("_c3", "session_id")
df_test = df_test.withColumnRenamed("_c4", "action")
df_test = df_test.withColumnRenamed("_c5", "object")
df_test = df_test.withColumnRenamed("_c6", "time")
df_test = df_test.where(df_test.username != 'username')

In [3]:
df_train.show(n=5)

+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|enroll_id|username|           course_id|          session_id|     action|              object|               time|
+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...|click_about|                null|2015-09-27T15:42:59|
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...| click_info|                null|2015-09-27T15:43:12|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...|pause_video|3dac5590435e43b3a...|2015-10-19T19:37:42|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| load_video|3dac5590435e43b3a...|2015-10-19T19:33:27|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| play_video|3dac5590435e43b3a...|2015-10-19T19:33:30|
+---------+--------+--------------------+--------------------+----------

In [5]:
print((df_train.count(), len(df_train.columns)))

(29165540, 7)


In [4]:
df_test.show(n=5)

+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|enroll_id|username|           course_id|          session_id|     action|              object|               time|
+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|      775| 1520977|course-v1:Tsinghu...|5f421f644193c2d48...| load_video|3dac5590435e43b3a...|2015-10-15T22:14:11|
|      775| 1520977|course-v1:Tsinghu...|5f421f644193c2d48...| load_video|3169d758ee2d4262b...|2015-10-15T22:43:35|
|      775| 1520977|course-v1:Tsinghu...|5f421f644193c2d48...| play_video|3169d758ee2d4262b...|2015-10-15T22:43:40|
|      775| 1520977|course-v1:Tsinghu...|5f421f644193c2d48...|pause_video|3169d758ee2d4262b...|2015-10-15T22:55:38|
|      775| 1520977|course-v1:Tsinghu...|5f421f644193c2d48...| stop_video|3169d758ee2d4262b...|2015-10-15T22:55:38|
+---------+--------+--------------------+--------------------+----------

In [8]:
print((df_test.count(), len(df_test.columns)))

(12944862, 7)


In [9]:
df_combined = df_train.union(df_test)

In [10]:
df_combined.show(n=10)

+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|enroll_id|username|           course_id|          session_id|     action|              object|               time|
+---------+--------+--------------------+--------------------+-----------+--------------------+-------------------+
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...|click_about|                null|2015-09-27T15:42:59|
|      772|    5981|course-v1:Tsinghu...|d8a9b787fa69063c3...| click_info|                null|2015-09-27T15:43:12|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...|pause_video|3dac5590435e43b3a...|2015-10-19T19:37:42|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| load_video|3dac5590435e43b3a...|2015-10-19T19:33:27|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...| play_video|3dac5590435e43b3a...|2015-10-19T19:33:30|
|      773| 1544995|course-v1:Tsinghu...|2f02b86eb3ea2cbf0...|pause_vide

In [11]:
print((df_combined.count(), len(df_combined.columns)))

(42110402, 7)


## Análise exploratória

In [11]:
df_combined.select('action').distinct().collect()

[Row(action='close_courseware'),
 Row(action='play_video'),
 Row(action='click_about'),
 Row(action='click_forum'),
 Row(action='problem_check_incorrect'),
 Row(action='create_thread'),
 Row(action='seek_video'),
 Row(action='problem_check_correct'),
 Row(action='reset_problem'),
 Row(action='create_comment'),
 Row(action='load_video'),
 Row(action='problem_get'),
 Row(action='pause_video'),
 Row(action='delete_comment'),
 Row(action='problem_check'),
 Row(action='click_courseware'),
 Row(action='close_forum'),
 Row(action='stop_video'),
 Row(action='delete_thread'),
 Row(action='problem_save'),
 Row(action='click_progress'),
 Row(action='click_info')]

In [12]:
df_combined.createOrReplaceTempView("table1")

In [19]:
query = spark.sql("SELECT action, COUNT(action) AS quantidade FROM table1 GROUP BY action ORDER BY quantidade DESC")

In [20]:
query.collect()

[Row(action='stop_video', quantidade=8424429),
 Row(action='pause_video', quantidade=6017142),
 Row(action='click_courseware', quantidade=5685535),
 Row(action='play_video', quantidade=4895238),
 Row(action='problem_get', quantidade=3291858),
 Row(action='load_video', quantidade=3039857),
 Row(action='close_courseware', quantidade=2890604),
 Row(action='seek_video', quantidade=2574885),
 Row(action='problem_check', quantidade=1219175),
 Row(action='click_info', quantidade=1043535),
 Row(action='problem_check_correct', quantidade=935239),
 Row(action='click_about', quantidade=720867),
 Row(action='problem_check_incorrect', quantidade=499470),
 Row(action='click_progress', quantidade=282822),
 Row(action='click_forum', quantidade=269136),
 Row(action='problem_save', quantidade=241272),
 Row(action='create_comment', quantidade=49958),
 Row(action='reset_problem', quantidade=18464),
 Row(action='create_thread', quantidade=9200),
 Row(action='delete_comment', quantidade=1277),
 Row(action='

In [24]:
query = spark.sql("SELECT username, COUNT(action) AS quantidade_de_actions FROM table1 GROUP BY username ORDER BY quantidade_de_actions ASC");

In [25]:
query.collect()

[Row(username='3116768', quantidade_de_actions=1),
 Row(username='673161', quantidade_de_actions=1),
 Row(username='1507431', quantidade_de_actions=1),
 Row(username='3636828', quantidade_de_actions=1),
 Row(username='2802895', quantidade_de_actions=1),
 Row(username='2759108', quantidade_de_actions=1),
 Row(username='225272', quantidade_de_actions=1),
 Row(username='300513', quantidade_de_actions=1),
 Row(username='4712999', quantidade_de_actions=1),
 Row(username='5214806', quantidade_de_actions=1),
 Row(username='522183', quantidade_de_actions=1),
 Row(username='134860', quantidade_de_actions=1),
 Row(username='439928', quantidade_de_actions=1),
 Row(username='62954', quantidade_de_actions=1),
 Row(username='835413', quantidade_de_actions=1),
 Row(username='530360', quantidade_de_actions=1),
 Row(username='116137', quantidade_de_actions=1),
 Row(username='727035', quantidade_de_actions=1),
 Row(username='4346460', quantidade_de_actions=1),
 Row(username='375115', quantidade_de_actio

In [75]:
query = spark.sql("""SELECT
  CASE
    WHEN quantidade_de_actions < 10 THEN 'Menos de 10'
    WHEN quantidade_de_actions BETWEEN 10 AND 100 THEN 'Entre 10 e 100'
    WHEN quantidade_de_actions BETWEEN 100 AND 250 THEN 'Entre 100 e 250'
    WHEN quantidade_de_actions BETWEEN 250 AND 500 THEN 'Entre 250 e 500'
    WHEN quantidade_de_actions BETWEEN 500 AND 1000 THEN 'Entre 500 e 1000'
    WHEN quantidade_de_actions BETWEEN 1000 AND 2500 THEN 'Entre 1000 e 2500'
    WHEN quantidade_de_actions BETWEEN 2500 AND 5000 THEN 'Entre 2500 e 5000'
    WHEN quantidade_de_actions > 5000 THEN 'Mais de 5000'
  END AS faixa_actions,
  COUNT(*) AS quantidade_de_usuarios
FROM (
  SELECT username, COUNT(action) AS quantidade_de_actions
  FROM table1
  GROUP BY username
) AS subquery
GROUP BY faixa_actions;""")

In [76]:
query.collect()

[Row(faixa_actions='Entre 250 e 500', quantidade_de_usuarios=9893),
 Row(faixa_actions='Entre 10 e 100', quantidade_de_usuarios=26924),
 Row(faixa_actions='Entre 1000 e 2500', quantidade_de_usuarios=8467),
 Row(faixa_actions='Entre 100 e 250', quantidade_de_usuarios=13579),
 Row(faixa_actions='Entre 500 e 1000', quantidade_de_usuarios=8969),
 Row(faixa_actions='Mais de 5000', quantidade_de_usuarios=777),
 Row(faixa_actions='Menos de 10', quantidade_de_usuarios=6412),
 Row(faixa_actions='Entre 2500 e 5000', quantidade_de_usuarios=2062)]

In [77]:
query = spark.sql("""
SELECT
  CASE
    WHEN quantidade_de_actions < 10 THEN 'Menos de 10'
    WHEN quantidade_de_actions BETWEEN 10 AND 100 THEN 'Entre 10 e 100'
    WHEN quantidade_de_actions BETWEEN 100 AND 250 THEN 'Entre 100 e 250'
    WHEN quantidade_de_actions BETWEEN 250 AND 500 THEN 'Entre 250 e 500'
    WHEN quantidade_de_actions BETWEEN 500 AND 1000 THEN 'Entre 500 e 1000'
    WHEN quantidade_de_actions BETWEEN 1000 AND 2500 THEN 'Entre 1000 e 2500'
    WHEN quantidade_de_actions BETWEEN 2500 AND 5000 THEN 'Entre 2500 e 5000'
    WHEN quantidade_de_actions > 5000 THEN 'Mais de 5000'
  END AS faixa_actions,
  COUNT(*) AS quantidade_de_usuarios
FROM (
  SELECT username, COUNT(action) AS quantidade_de_actions
  FROM table1
  WHERE action in ('problem_get', 'problem_check_correct', 'problem_check', 'problem_check_incorrect', 'problem_save')
  GROUP BY username
) AS subquery
GROUP BY faixa_actions;
""")

In [78]:
query.collect()

[Row(faixa_actions='Entre 250 e 500', quantidade_de_usuarios=3874),
 Row(faixa_actions='Entre 10 e 100', quantidade_de_usuarios=24634),
 Row(faixa_actions='Entre 1000 e 2500', quantidade_de_usuarios=316),
 Row(faixa_actions='Entre 100 e 250', quantidade_de_usuarios=10121),
 Row(faixa_actions='Entre 500 e 1000', quantidade_de_usuarios=1383),
 Row(faixa_actions='Mais de 5000', quantidade_de_usuarios=33),
 Row(faixa_actions='Menos de 10', quantidade_de_usuarios=14274),
 Row(faixa_actions='Entre 2500 e 5000', quantidade_de_usuarios=59)]

In [67]:
query = spark.sql("""
    SELECT
    DATE_FORMAT(`time`, 'yyyy MMM') AS mes_ano,
    COUNT(*) AS total_de_actions
    FROM table1
    GROUP BY mes_ano
    ORDER BY mes_ano ASC;
""")


In [68]:
query.collect()

[Row(mes_ano='2015 Aug', total_de_actions=76339),
 Row(mes_ano='2015 Dec', total_de_actions=219480),
 Row(mes_ano='2015 Jul', total_de_actions=378856),
 Row(mes_ano='2015 Jun', total_de_actions=249967),
 Row(mes_ano='2015 Nov', total_de_actions=1370320),
 Row(mes_ano='2015 Oct', total_de_actions=8261446),
 Row(mes_ano='2015 Sep', total_de_actions=4668510),
 Row(mes_ano='2016 Apr', total_de_actions=1936421),
 Row(mes_ano='2016 Aug', total_de_actions=332799),
 Row(mes_ano='2016 Feb', total_de_actions=1337815),
 Row(mes_ano='2016 Jan', total_de_actions=76552),
 Row(mes_ano='2016 Jul', total_de_actions=3640108),
 Row(mes_ano='2016 Jun', total_de_actions=1012335),
 Row(mes_ano='2016 Mar', total_de_actions=8059035),
 Row(mes_ano='2016 May', total_de_actions=1068757),
 Row(mes_ano='2016 Nov', total_de_actions=129188),
 Row(mes_ano='2016 Oct', total_de_actions=2665433),
 Row(mes_ano='2016 Sep', total_de_actions=2940131),
 Row(mes_ano='2017 Apr', total_de_actions=50991),
 Row(mes_ano='2017 Feb'

In [70]:
query = spark.sql("""
    SELECT
    mes_ano,
    AVG(quantidade_de_actions) AS media_de_actions_por_usuario
FROM
    (
        SELECT
            DATE_FORMAT(time, 'yyyy MMM') AS mes_ano,
            username,
            COUNT(*) AS quantidade_de_actions
        FROM
            table1
        GROUP BY
            mes_ano,
            username
    ) AS subquery
GROUP BY
    mes_ano
SORT BY mes_ano ASC
""")

In [71]:
query.collect()

[Row(mes_ano='2015 Aug', media_de_actions_por_usuario=120.59873617693523),
 Row(mes_ano='2015 Dec', media_de_actions_por_usuario=96.72983693256941),
 Row(mes_ano='2015 Jul', media_de_actions_por_usuario=182.1423076923077),
 Row(mes_ano='2015 Jun', media_de_actions_por_usuario=131.63085834649814),
 Row(mes_ano='2015 Nov', media_de_actions_por_usuario=161.84244714775008),
 Row(mes_ano='2015 Oct', media_de_actions_por_usuario=322.59932055136863),
 Row(mes_ano='2015 Sep', media_de_actions_por_usuario=233.64746509183723),
 Row(mes_ano='2016 Apr', media_de_actions_por_usuario=168.2673792144595),
 Row(mes_ano='2016 Aug', media_de_actions_por_usuario=134.5729882733522),
 Row(mes_ano='2016 Feb', media_de_actions_por_usuario=172.39884020618555),
 Row(mes_ano='2016 Jan', media_de_actions_por_usuario=52.649243466299865),
 Row(mes_ano='2016 Jul', media_de_actions_por_usuario=287.41476510067116),
 Row(mes_ano='2016 Jun', media_de_actions_por_usuario=256.93781725888323),
 Row(mes_ano='2016 Mar', medi

In [48]:
df_combined.groupBy('action').count().orderBy('count', ascending=False).show(n=22)

+--------------------+-------+
|              action|  count|
+--------------------+-------+
|          stop_video|8424429|
|         pause_video|6017142|
|    click_courseware|5685535|
|          play_video|4895238|
|         problem_get|3291858|
|          load_video|3039857|
|    close_courseware|2890604|
|          seek_video|2574885|
|       problem_check|1219175|
|          click_info|1043535|
|problem_check_cor...| 935239|
|         click_about| 720867|
|problem_check_inc...| 499470|
|      click_progress| 282822|
|         click_forum| 269136|
|        problem_save| 241272|
|      create_comment|  49958|
|       reset_problem|  18464|
|       create_thread|   9200|
|      delete_comment|   1277|
|       delete_thread|    434|
|         close_forum|      5|
+--------------------+-------+



# Plotagem e Sumarização